##Download dataset

In [1]:
import transformers

In [2]:
import pandas as pd
data=pd.read_csv("all_data.csv")

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
print(data.keys())


Index(['Unnamed: 0', 'paper', 'slide'], dtype='object')


In [5]:
#remove extra column
data=data.drop(['Unnamed: 0'],axis=1)

In [6]:
print(data.keys())


Index(['paper', 'slide'], dtype='object')


In [7]:
print(data.head())

                                               paper  \
0   1  Abstract Voice over IP (VoIP) systems are ...   
1  TOPHAT:Topology-basedHost-LevelAttributionforM...   
2  Sirius:Probabilisticdataassertionsfordetecting...   
3    1SLAM: Sleep-Wake Aware Local Monitoring in ...   
4  PRIVATUS:Wallet-FriendlyPrivacyProtectionforSm...   

                                               slide  
0  Slide 1/18DCSL: Dependable Computing Systems L...  
1  10/26/20171TOPHAT: Topology based Host‐Level A...  
2  Slide 1/27Sirius: Neural network based probabi...  
3  Slide 1/27DCSL: Dependable Computing Systems L...  
4  Slide 1/29  PRIVATUS: Wallet-Friendly Privacy ...  


In [8]:
print(len(data['paper'][0]))

46376


## Installation of transformers and checking the GPU 

In [9]:
# !pip install --quiet transformers==2.9.0
# !nvidia-smi

## Necessary Imports and model download

In [10]:
import random
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)
def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
set_seed(42)

## Initialize the mode and its tokenizer

In [12]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')
t5_model = T5ForConditionalGeneration.from_pretrained('t5-base')

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

## Initialize the optimizer

In [13]:
# optimizer
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in t5_model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
    {
        "params": [p for n, p in t5_model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
optimizer = AdamW(optimizer_grouped_parameters, lr=3e-4, eps=1e-8)

In [20]:
torch.cuda.is_available()
device=torch.device("cuda:0")
t5_model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dr

In [50]:
ready_data =[]
for i,r in data.iterrows():
    input=r['paper']
    output=r['slide']
    Paper = "summary: "+input+ " </s>"
    slide = output+" </s>"
     
    tokenized_inp = tokenizer.encode_plus(Paper,  max_length=100, pad_to_max_length=True,return_tensors="pt")
    tokenized_output = tokenizer.encode_plus(slide, max_length=100, pad_to_max_length=True,return_tensors="pt")
     
    input_ids  = tokenized_inp["input_ids"]
    attention_mask = tokenized_inp["attention_mask"]
    
    lm_labels= tokenized_output["input_ids"]
    decoder_attention_mask=  tokenized_output["attention_mask"]
  
    ready_data.append([np.array(input_ids , dtype='float32' ) ,np.array(attention_mask , dtype='float32' ), np.array(lm_labels, dtype='float32' ) ,np.array(decoder_attention_mask, dtype='float32' )])

In [51]:
print(len(ready_data))
print(len(ready_data[0]))
ready0= np.array(ready_data )
print(ready0.shape)
ready = torch.FloatTensor(np.stack(ready0))

406
4
(406, 4, 1, 100)


In [ ]:
ready.to(device)
dataloader 

## Training the mode

In [25]:
t5_model.train()

epochs = 1

for epoch in range(epochs):
  print ("epoch ",epoch)
  for i,r in data.iterrows():
    input=r['paper']
    output=r['slide']
    Paper = "summary: "+input+ " </s>"
    slide = output+" </s>"
    
    tokenized_inp = tokenizer.encode_plus(Paper,  max_length=100, pad_to_max_length=True,return_tensors="pt")
    tokenized_output = tokenizer.encode_plus(slide, max_length=100, pad_to_max_length=True,return_tensors="pt")
    
    tokenized_inp.to(device)
    tokenized_output.to(device)
    
    input_ids  = tokenized_inp["input_ids"]
    attention_mask = tokenized_inp["attention_mask"]
    
    lm_labels= tokenized_output["input_ids"]
    decoder_attention_mask=  tokenized_output["attention_mask"]
  
    input_ids.to(device)
    lm_labels.to(device)
    decoder_attention_mask.to(device)
    attention_mask.to(device)
    
    # the forward function automatically creates the correct decoder_input_ids
    expected_output = t5_model(input_ids=input_ids, labels=lm_labels,decoder_attention_mask=decoder_attention_mask,attention_mask=attention_mask )
    loss = expected_output[0]

    loss.backward()
    optimizer.step()

epoch  0


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

## Testing the model

In [ ]:

test_sent = data['paper'][0]+ " </s>"
test_tokenized = tokenizer.encode_plus(test_sent, return_tensors="pt")

test_input_ids  = test_tokenized["input_ids"]
test_attention_mask = test_tokenized["attention_mask"]

t5_model.eval()
beam_outputs = t5_model.generate(
    input_ids=test_input_ids,attention_mask=test_attention_mask,
    max_length=100,
    early_stopping=True,
    num_beams=10,
    num_return_sequences=3,
    no_repeat_ngram_size=2
)

for beam_output in beam_outputs:
    sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
    print (sent)



NameError: ignored